In [24]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/bank_churn.csv')
dataset=dataset.dropna()
#dataset=dataset.fillna(0)

#transform data into numeric value
dataset['Geography']=le.fit_transform(dataset['Geography'])
dataset['Gender']=le.fit_transform(dataset['Gender'])

#X=dataset.drop(['Exited','CustomerId','RowNumber','Surname'],axis=1)
#y=dataset["Exited"]

dataset=dataset.dropna()
dataset=dataset.fillna(0)

dataset=dataset.drop(['RowNumber','CustomerId','Surname'],axis=1)
dataset['Geography']=le.fit_transform(dataset['Geography'])
dataset['Gender']=le.fit_transform(dataset['Gender'])

df1=dataset[dataset['Exited']==0]
df=dataset[dataset['Exited']==1]


In [ ]:
X

NameError: name 'X' is not defined

In [25]:
pip install ctganenn

In [26]:
from ctganenn import CTGANENN

X,y=CTGANENN(df,df1,5926,'Exited')

Gen. (-0.87) | Discrim. (-0.03): 100%|██████████| 100/100 [01:09<00:00,  1.44it/s]


In [29]:
# SVM
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
#balance = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
#param_grid = dict()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.744310 using {'class_weight': {0: 100, 1: 100}}
0.742081 (0.010050) with: {'class_weight': {0: 100, 1: 10}}
0.738987 (0.010909) with: {'class_weight': {0: 100, 1: 20}}
0.735925 (0.010847) with: {'class_weight': {0: 100, 1: 30}}
0.736024 (0.011676) with: {'class_weight': {0: 100, 1: 40}}
0.738480 (0.011946) with: {'class_weight': {0: 100, 1: 50}}
0.739808 (0.011974) with: {'class_weight': {0: 100, 1: 60}}
0.741015 (0.012365) with: {'class_weight': {0: 100, 1: 70}}
0.742143 (0.012343) with: {'class_weight': {0: 100, 1: 80}}
0.743256 (0.012074) with: {'class_weight': {0: 100, 1: 90}}
0.744310 (0.011847) with: {'class_weight': {0: 100, 1: 100}}


In [30]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('svm-bank-ctganenn-cs-auc-fn.xlsx', index=False)

In [31]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.765771 using {'class_weight': {0: 100, 1: 100}}
0.519291 (0.009586) with: {'class_weight': {0: 100, 1: 10}}
0.544555 (0.012110) with: {'class_weight': {0: 100, 1: 20}}
0.571667 (0.010226) with: {'class_weight': {0: 100, 1: 30}}
0.595823 (0.014717) with: {'class_weight': {0: 100, 1: 40}}
0.650419 (0.025566) with: {'class_weight': {0: 100, 1: 50}}
0.711249 (0.015198) with: {'class_weight': {0: 100, 1: 60}}
0.739890 (0.008384) with: {'class_weight': {0: 100, 1: 70}}
0.752177 (0.006520) with: {'class_weight': {0: 100, 1: 80}}
0.759847 (0.007115) with: {'class_weight': {0: 100, 1: 90}}
0.765771 (0.007260) with: {'class_weight': {0: 100, 1: 100}}


In [32]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('svm-bank-ctganenn-cs-f1-fn.xlsx', index=False)

In [33]:
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score, make_scorer
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.687895 using {'class_weight': {0: 100, 1: 100}}
0.322729 (0.016352) with: {'class_weight': {0: 100, 1: 10}}
0.365307 (0.019557) with: {'class_weight': {0: 100, 1: 20}}
0.408237 (0.015768) with: {'class_weight': {0: 100, 1: 30}}
0.445296 (0.021680) with: {'class_weight': {0: 100, 1: 40}}
0.524377 (0.036238) with: {'class_weight': {0: 100, 1: 50}}
0.609603 (0.021660) with: {'class_weight': {0: 100, 1: 60}}
0.650018 (0.012164) with: {'class_weight': {0: 100, 1: 70}}
0.667850 (0.008755) with: {'class_weight': {0: 100, 1: 80}}
0.678913 (0.009375) with: {'class_weight': {0: 100, 1: 90}}
0.687895 (0.009831) with: {'class_weight': {0: 100, 1: 100}}


In [34]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('svm-bank-ctganenn-cs-gmean-fn.xlsx', index=False)